In [2]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd

In [3]:
ee.Initialize()

# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')
africa_bbox = ee.Geometry.Rectangle(-15.64, -33.58, 59.06, 25.96)
filtered = countries_gaul.filterBounds(africa_bbox)

In [9]:
subregions = {
  'na': 'Northern Africa',
  'ma': 'Middle Africa',
  'wa': 'Western Africa', 
  'ea': 'Eastern Africa',
  'sa': 'Southern Africa', 
  'ws': 'Western Asia'
}
subregions['ws']

'Western Asia'

In [ ]:
# AREA OF INTEREST
region = [[-25.0, -40.0], [65.0, -40.0], [65.0, 40.0], [-30.0, 40.0], [-30.0, -40.0]]
country_number = filtered.size().getInfo()

In [4]:
wpbm_stats = ee.Image('projects/fao-wapor/T_Annual/T_Annual-2010');

In [5]:
def vizualize_image(wbpm,regione_plot,scala):    
    visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region": regione_plot,
             "scale" : scala}
    
    url_WPbm = wbpm.getThumbUrl(visparams)
    return url_WPbm

In [6]:
%matplotlib inline  
Image(url=vizualize_image(wpbm_stats,region,250))

#### Means

In [9]:
means = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = 250,
)

#### Min Max

In [ ]:
minMaxs = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.minMax(),
    scale = 250
)

#### MinMax Sum combined

In [ ]:
reducers = ee.Reducer.minMax().combine(
    reducer2 = ee.Reducer.sum(),
    sharedInputs = True
);

minMaxStds = wpbm_stats.reduceRegions(
            collection = filtered,
            reducer = reducers,
            scale = 250,
);

## Export

In [ ]:
features_m = means.getInfo()['features']
features_mms = minMaxStds.getInfo()['features']

In [ ]:
features_mms

In [ ]:
df_m = pd.DataFrame(data=features_m[1:], columns = features_m[0])
df_mms = pd.DataFrame(data=features_mms[1:], columns = features_mms[0])

In [ ]:
serie_m = df_m['properties'].apply(pd.Series)
serie_mms = df_mms['properties'].apply(pd.Series)

In [ ]:
serie_m.columns

In [ ]:
serie_mms.columns

In [ ]:
serie_m['name'].head() # anche serie_m.name.head()

In [ ]:
serie_m['iso3'].head()

In [ ]:
serie_m['mean'].head() # non funziona con serie_m.mean.head() vedi sotto

In [ ]:
serie_m.mean().head()

In [ ]:
df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])

In [ ]:
df_mms = pd.DataFrame(serie_mms[['iso3', 'min', 'max','sum']])

In [ ]:
df_stats = df_m.join(df_mms, lsuffix='_df_m', rsuffix='_df_mms')

In [ ]:
df_stats

In [ ]:
df_stats.to_csv('2015_transp.csv')